# Yelping and Gender

In an earlier exercise we started to look at the following question:

  > Yelp wants to create the best user experience possible and show authentic reviews.  They have a proprietary 
  > algorithm for ranking the reviews they show users.  The average star rating for a business is part of it,
  > but not the whole story.  The number of reviews is part of it, but not the whole story. 

  > Since many users will not read more than a couple reviews, having a good algorithm for ordering the reviews to 
  > show them to a user is critical to Yelp's business.  They need to always be thinking of how to make the ranking better
  > in order to improve the customer experience.

  > What if men and women review differently?  Is a 4-star rating from a man the same as a 4-star rating from a woman?
  > In other words, might men or women consistently rate businesses higher or lower?  Would this depend on the type
  > of business?  Are ratings by one gender more consistent than the other?  If so, could we be more certain of the validity
  > of the ranking of a business based on 5 reviews by women than we would by 5 reviews by men (or vice versa)?

In that prior exercise we looked at data from the from the <a href="https://www.ssa.gov/oact/babynames/limits.html" target="_blank">Social Security Administration's website</a> to predict the 
gender of a name - is it male or female and how often?  

In this exercise we will match that data with Yelp user data to see how many Yelpers we can predict a gender for.
The start of this notebook repeats the work we did in that earlier notebook.  In your team's project, that would 
not be a separate notebook since it's part of the same data story.
# Part 1: Loading Data

### Step 1: SSA gender ratios
<div style="text-align:center;"><span style="color: red;"><strong>** WARNING **</strong></span></div>
In your team's project, a cell should **NOT** look like the following - you should have the commented steps in separate cells and documented with markdown.
<div style="text-align:center;"><span style="color: red;"><strong>** WARNING **</strong></span></div>

In [0]:
import pyspark.sql.functions as f

# load the data
df_ssa = spark.read.option("header","false").option("inferSchema","true").csv("/ssa/data/*.txt").withColumn("filename", f.input_file_name())
# add column headings
df_ssa2 = df_ssa.toDF("name","gender","cnt","filename")
# set the ssa_year column
df_ssa3 = df_ssa2.select(df_ssa2.name, df_ssa2.gender, df_ssa2.cnt, df_ssa2.filename.substr(-8,4).cast("integer").alias("ssa_year"))
# pivot to M /F columns
df_ssa4 = df_ssa3.groupBy("ssa_year","name").pivot("gender",["F","M"]).sum("cnt")
# create one line for each name
df_ssa5 = df_ssa4.groupBy("name").sum("F", "M").withColumnRenamed("sum(F)","women").withColumnRenamed("sum(M)","men")
# assign gender to a name
df_ssa6 = df_ssa5.select("name", "women", "men", \
               f.when(df_ssa5.women.isNull(),df_ssa5.men).when(df_ssa5.men.isNull(),df_ssa5.women)\
               .otherwise(df_ssa5.women + df_ssa5.men).alias("total"), \
               f.when(df_ssa5.men.isNull(),1).when(df_ssa5.women.isNull(),2).\
               when(df_ssa5.men > df_ssa5.women, 2).otherwise(1).alias("gender"))
# calculate a gender ratio for each name
df_ssaGender = df_ssa6.select("name", "women", "men", "total", "gender", \
                              f.when(df_ssa6.gender == 1, df_ssa6.women / df_ssa6.total).\
                              otherwise(df_ssa6.men / df_ssa6.total).alias("gender_ratio")).cache()
df_ssaGender.show()

+--------+-----+----+-----+------+------------------+
 name|women| men|total|gender| gender_ratio|
+--------+-----+----+-----+------+------------------+
 Soni| 310|null| 310| 1| 1.0|
 Lorne| 50|4263| 4313| 2|0.9884071412010201|
 Bonnye| 364|null| 364| 1| 1.0|
 Jaydn| 377| 649| 1026| 2|0.6325536062378168|
Gladiola| 166|null| 166| 1| 1.0|
 Faye|68929| 518|69447| 1|0.9925410744884589|
 Nimrit| 99|null| 99| 1| 1.0|
 Normand| null|5831| 5831| 2| 1.0|
Thurston| null|3445| 3445| 2| 1.0|
 Dhyey| null| 38| 38| 2| 1.0|
 Shian| 419| 6| 425| 1|0.9858823529411764|
 Kiana|24502| 19|24521| 1|0.9992251539496758|
 Tremone| null| 28| 28| 2| 1.0|
 Guyon| null| 10| 10| 2| 1.0|
 Sammye| 723|null| 723| 1| 1.0|
 Aryiel| 23|null| 23| 1| 1.0|
 Maxxon| null| 92| 92| 2| 1.0|
Elliahna| 59|null| 59| 1| 1.0|
 Susanna|18950|null|18950| 1| 1.0|
 Xhaiden| null| 135| 135| 2| 1.0|
+--------+-----+----+-----+------+------------------+
only showing top 20 rows

### Step 2: Load the Yelp User Data

To predict the gender of the Yelp users we need to load the Yelp user data.  In the following cell, 
we load the Yelp user data ito a DataFrame named `df_users`. We did this in a prior exercise, so here 
we are also going to include the `select` method from PySpark to get only the columns we want.

Keep in mind that the `user` data is larger than the business data, so you previously ran a notebook that built Spark tables for the user and review data files.


#### Using multiple steps
In the code shown here, we are building our DataFrame we want to use through multiple steps.  We could combine 
the steps, but we will see this approach of breaking steps out can be advantageous if we want to profile our transformations.  If we wanted
to combine these steps, one issue we run into is that when we go to split the column, we would be splitting a column
in a DataFrame that has no name since it was generated as an intermediate step, so how could refer to that field?  One alternative is to use the `column` function
which allows a PySpark formula to refer to a field in an intermediate DataFrame by name, such as `f.column("elite")` 
to refer to the `elite` column. The `f` in this case assumes that `pyspark.sql.functions` was already imported as `f`.

#### Calculating the years elite
The `elite` filed is a comma-seperated list of the years a user is elite, so we are going to split the `elite` field the same way we split the `categories` field
in the business data in an earlier exercise.  However, for this notebook we don't need the actual years a user is elite, but only
the total number of years they are elite.  To calculate that, the `split` function is used to create an array
from a string, and then the `size` function is used to get the size of that array.


Be sure to also print the record count - you should have 1,968,703 users.

In [0]:
if ( spark.catalog._jcatalog.tableExists("users_table") ):
  print("Loading user data from table ...")
  df_userData0 = spark.read.format('parquet').table("users_table")
  print("Loaded user data from table")
else:
  print("The Building Review and User Tables notebook has not been run, loading from files ...")
  df_userData0 = spark.read.json("/yelp/user.bz2").withColumn("friend_count",f.size(f.split(f.col("friends"),'\s*,\s*'))).drop("friends")
  # Does not already exist as a table, so write it out
  df_userData0.write.mode("overwrite").format('parquet').saveAsTable("users_table")
  print("Loaded user data from file")

Loading user data from table ...
Loaded user data from table

In [0]:
df_userData1 = df_userData0.select("user_id", "name", "review_count", "elite", "yelping_since")
df_userData1.createOrReplaceTempView("userData1")
df_users = spark.sql("""
SELECT user_id, name, review_count, yelping_since, 
       IF( LENGTH(elite) = 0, 0, SIZE( SPLIT(elite, '\s*,\s*') ) ) AS years_elite
FROM userData1
""").cache()

print("number of users:", df_users.count())
df_users.printSchema()
df_users.show(100, truncate=False)

number of users: 1968703
root
-- user_id: string (nullable = true)
-- name: string (nullable = true)
-- review_count: long (nullable = true)
-- yelping_since: string (nullable = true)
-- years_elite: integer (nullable = false)

+----------------------+---------+------------+-------------------+-----------+
user_id |name |review_count|yelping_since |years_elite|
+----------------------+---------+------------+-------------------+-----------+
Qw_H_0xUHJN1eybBmbhZsA|Ken |4 |2017-12-09 19:50:44|0 |
E3K2qQ0bkWrSyiikh4OnjA|Brandon |23 |2016-03-11 21:53:39|0 |
Ba_mM1i4OAuZ57MVYsLSwQ|Rick |1 |2018-01-10 16:13:55|0 |
GRujMvFN83-IBQAwTkteLQ|B.v. |13 |2018-02-26 20:42:47|0 |
b4YX1azbMwgV5VLu3lrpiA|Rahul |1 |2018-02-24 07:36:13|0 |
pbenW4yj10LaugmC7xDTGQ|Michelle |6 |2013-12-05 01:41:12|0 |
x2fKuZV85FvrSxH-ob04iQ|David |8 |2018-01-19 19:32:03|0 |
Ru7u8IMunZswWbQZDJl4eA|Genesis |1 |2018-01-23 20:14:30|0 |
QB1Ds4_-Dd48ho5pM5r_SQ|Stephanie|11 |2018-03-29 01:47:33|0 |
cfFS07Gzi5-xSWkQj5271g|Anthony |1 |2018-04-23 16:26:07|0 |
uxCTAXR2b5j_UY3PYApoRw|Alexandre|2 |2018-03-26 16:46:03|0 |
x0OHPeQSueJYBEde1Pp1uA|Eric |1 |2016-03-15 02:12:43|0 |
uIPRJrQ8RI1iRG0f5qcSPw|Katie |3 |2018-05-09 19:58:58|0 |
Exps1-qakisEggqNsKSwFg|Dianne |2 |2015-06-03 18:07:28|0 |
W3AFSt-zVgBl-8kCVOvWNQ|Stephanie|18 |2014-06-17 03:58:21|0 |
Kv2s1OlVuq0wYlnc_UvNPA|Lauren |30 |2017-12-30 00:04:41|0 |
LjWmAmykgZYHXDa_QOpBLw|Anne |7 |2008-11-03 23:46:58|0 |
lHMgyE4hF18kROw-xgA17w|Kesha |1 |2018-07-25 18:14:05|0 |
gjAYV29Ln0EnNEVTTYwbJg|Jahkey |2 |2016-06-05 04:38:16|0 |
jCy-JsEKBaU4wSKoYXC7hQ|Jackie |3 |2018-06-02 20:48:58|0 |
gfKcR1JAXzHRtC7jhqlBAQ|Xiaohui |4 |2013-11-18 07:16:22|0 |
KA_gclZmAa0_yG-6nW4VFw|Karen |6 |2012-07-24 21:34:48|0 |
WD9CRqsW0bcBPPMwlaN1fw|Sydney |2 |2017-05-10 07:28:19|0 |
4492Oe7UTXnOKvzaS0GqgQ|Nicole |5 |2017-06-17 03:00:30|0 |
DvQHLD-nDnknKxPzGK2PDQ|Angelia |7 |2011-01-16 07:15:49|0 |
JMXx9V6FpPHIvlTNiWKtNg|Sam |3 |2014-03-16 03:04:31|0 |
M-2l4n1We9m8k4TQGRekAw|Marina |5 |2014-07-02 02:47:59|0 |
uiB2cQ3aAtwkLQy5pCmgLw|Steven |3 |2013-09-21 20:03:32|0 |
7khnwCRzK8mmMeTtJtwQ1Q|Mario |5 |2017-05-31 04:34:14|0 |
L9uHpfbbATJMmhjybUQjgw|Keith |11 |2011-03-24 15:32:46|0 |
8l4XkJVyAV3Zjqa093BpKw|Enrique |70 |2012-02-19 19:22:35|1 |
m4sp3_AaS3J0iKR4s73rYw|Ben |2 |2012-12-15 02:36:39|0 |
1RDeszhCu8V4mRQCQmhUKg|Leah |1 |2017-06-14 19:45:43|0 |
0ICagu_KwvaLu6nwn_15XA|Robert |2 |2017-07-10 04:12:41|0 |
JLusLQwdCF4kZ45415Jftg|Sam |3 |2017-06-26 23:07:18|0 |
N3s2FIg11P3ezCITIY0fbQ|Ginny |1 |2016-08-08 19:17:14|0 |
ijy-JOxm8-nWAxWZiCUtWQ|Rachad |2 |2017-07-05 00:48:30|0 |
otc-pXbG6ZJi18GqBxAfVg|Jeff |1 |2016-11-19 05:32:14|0 |
Z5iQWVMjgzaAQi6EqVJ2OA|Casey |1 |2017-08-21 23:30:23|0 |
KGsepM1qHpytSwBgP4_IAw|Skip |17 |2011-07-11 13:16:16|0 |
c3gwUtMgBKGAH_WmMi-GZg|Kevin |13 |2013-06-02 12:01:12|0 |
aIQkEn1E-Dryo2SB9I_6Hg|Christin |1 |2017-07-27 19:46:37|0 |
nGl5YnvPc1ppQTkXxBByZQ|Denita |12 |2017-04-27 17:14:42|0 |
Z7AdnbAHCyxu1CJV3jvzqw|Carson |4 |2012-06-07 00:26:34|0 |
CxAl4UXLGA7LNmaq8BADXg|Bubbley |1 |2017-09-09 18:03:55|0 |
EW17bnVZNVuCi-1X5USnfQ|Amy |4 |2014-07-12 01:24:41|0 |
Ry_JZj-_ILqinFJD_yIG9g|Brian |1 |2017-08-12 07:02:49|0 |
yqSzHFZ0p_u1IjoK6rVIbA|Damian |2 |2017-09-24 14:46:11|0 |
AYXpEG0QhLHxdQK_sTj6cg|Allyson |1 |2014-03-21 23:32:56|0 |
jdzBWd2Tl6tUiApuZPvAzQ|Jessa |1 |2017-09-24 21:29:23|0 |
W1bo-z9__mBPIzEowhhgvw|Lisa |13 |2014-11-27 16:56:43|0 |
IvrA9PXlCdPmc3C-VbSJnQ|Ashley |1 |2017-07-19 22:29:39|0 |
sRV6-CQlmz_3ZdNywK01bg|Martin |3 |2017-10-13 14:30:44|0 |
4XjH-QYfHJp60ydqTOd6wA|Steven |1 |2017-11-15 15:04:51|0 |
XJiyCG5Mb1qE8ZK7aHiT0w|DeNyus |2 |2015-04-18 00:19:49|0 |
7WNJP1YnloSRO0qM582FjA|Maddy |3 |2012-09-17 18:30:10|0 |
Dvk_7bEa4f49eCwvO86RlQ|Adam |1 |2017-11-20 20:26:12|0 |
p1B8pZ_7A03IIbc2a9a3XQ|Stephanie|2 |2016-01-17 18:19:09|0 |
41EslFdoRjwwF4sPCabu9g|Pranay |1 |2017-12-29 23:52:23|0 |
R3MNXjfLXGZmPS4DmPQ_bw|Garrett |2 |2016-08-14 17:58:35|0 |
7nwFTFNO5ECerGANRYPwLw|Regina |7 |2014-07-05 14:12:27|0 |
f49mZ4L3DdGV1OaLH-AZuA|Teresa |1

#### Step 2a: Profiling our transformation

When we do a transformation, it's often a good idea to check it.  It's not always the case that we would keep all of these checks inthe notebook, but
we want to check for errors before continuing.

In the next cell, we will get a count of the number of users who have been elite from zero years to the maximum number of years elite.  Since Yelp started in 2004, that puts a cap on how many years anyone can be elite.

In the empty code cell below, group the `df_users` DataFrame based on the years a user has been elite (`years_elite`), and then get a count.

In [0]:
# ADD YOUR PROFILING HERE
df_users.groupBy("years_elite").count().orderBy("years_elite").show()


+-----------+-------+
years_elite| count|
+-----------+-------+
 0|1892742|
 1| 18731|
 2| 20563|
 3| 13234|
 4| 8680|
 5| 4819|
 6| 3314|
 7| 2817|
 8| 1543|
 9| 1105|
 10| 657|
 11| 302|
 12| 133|
 13| 63|
+-----------+-------+

### Step 3: Create temporary views

We did this in a prior exercise.  Since we want to use the `.sql` method of the SparkSession (spark) to run a SQL 
query, we need to first create temporary views.  For the `df_users` DataFrame we create a temporary view named `users`.
For the `df_ssaGender` DataFrame we create a temporary view named `gender`.

In [0]:
df_users.createOrReplaceTempView("users")
df_ssaGender.createOrReplaceTempView("gender")

# Part 2: Assigning a gender to each user (or not)

We now want to use our SSA data to assign a gender to each user.  Will each Yelp user have a name we
can predict the gender for?  Probably not, but how big of an issue is it?

### Step 1: Bringing SSA gender data and Yelp user data together

We are going to do a join of our user data and the gender data:

> users JOIN gender

However, if we do an INNER JOIN, the resulting DataFrame will only include those users who
match with a corresponding name in the SSA data, so we want to do a LEFT OUTER JOIN so we include 
all of the rows from the view on the left (users).  When there is not a matching record from
the view on the right (gender), the columns selected from that view will be null for 
those rows.

Instead of bringing over the `gender` and `gender_ratio` fields from the `gender` view, we are 
going to bring these values over when they are not null, but set the `gender` to 3 when there is
no matching name (the 3 will indicate we don't know the gender of the name), and set the 
`gender_ratio` to 1.00 when there is no match (we are certain we don't know).  To do this we will use an
`IF` function in the query's `SELECT` clause:

`
 IF(gender IS NULL, 3, gender) AS gender
`
 
 In the following cell, write a `LEFT OUTER JOIN` query which gets all of the fields from the `users` view, 
 uses the above `IF` function to get the gender as `gender`, and include another `IF` function
 to get the gender_ratio (with the change above when it's null) as `gender_ratio`.
 
 The last issue we need to address is whether the user is elite.  Since users move in and out of elite status, 
 we need to decide what we mean by a "user is elite".  For purposes of this analysis, we will consider a user
 elite if they have been elite for any year in the data.  The reason we are doing that for this analysis is that 
 Yelp says an elite user must be tied to a real person, so if a user was *ever* elite, that would suggest
 they needed to enter their real name (or a more ralistic name than "Fast Food").  We already have the number 
 of years a user is elite, so we could add a field that checks if the number of years they have been elite exceeds zero.
 
 The call to the `sql` method will return a DataFrame; name it `df_user_gender`.
 
Also print the number of records - it should still match your user count from earlier.

#### Not sure how to proceed?  As MC Hammer says, "Break it Down":

* Every query has a `SELECT`  and `FROM` clause so write that
* You know the two views you want to join, so put them in the `FROM`clause and give them aliases
* You know you want to do a `LEFT OUTER JOIN` (assuming you listed the `users` view first), so put that in the `FROM` clause
* How are you going to match the names?  Put those fields from each view in the `ON` portion of the `FROM`clause
* Now we have to say what we want to select.  We want every field from the `users` view.  If you don't remember how to get all of the fields (and your teammates forgot too), just list all of the fields from the `users`view
* You want two other fields in your `SELECT` clause, the gender and gender ratio.  The `IF` function for the `gender` field is listed above
* Add a similar `IF` function to get the gender ratio
* Add a formula to calculate an `is_elite` filed with a boolean value (T/F) based on whether the `years_elite` field is greater than 0
* Don't forget to ***also print the record count*** and show a few records from the new DataFrame your SQL query generated

In [0]:
# Build Your Query
df_user_gender = spark.sql("""
select U.*, 
if(gender is null,3,gender) as gender,
if(gender is null,1.0,gender_ratio) as gender_ratio,
(years_elite>0) as is_elite
from users as u left outer join gender as G
on lower(u.name)=lower(G.name)



 """)

print("record count:", df_user_gender.count() )
df_user_gender.show(truncate=False)

record count: 1968703
+----------------------+-------+------------+-------------------+-----------+------+------------------+--------+
user_id |name |review_count|yelping_since |years_elite|gender|gender_ratio |is_elite|
+----------------------+-------+------------+-------------------+-----------+------+------------------+--------+
NXOiHZHfVwo3UORwuo0L4w|Jasmyn |7 |2017-10-15 01:56:40|0 |1 |1.0 |false |
O9b4JpDUM7BSizmP7HYpuA|Sassha |4 |2017-12-02 22:05:19|0 |3 |1.0 |false |
pj7i9k5cPBk7lUTLN4Xofg|Monte |1 |2017-09-02 20:56:22|0 |2 |0.9648698804540158|false |
IMSFx44J70qoyDWf8Bmbww|Trina |3 |2017-08-17 05:06:53|0 |1 |0.9990773205388448|false |
LExY_mffQNrDHf1_QR7opw|Merrie |3 |2018-04-01 17:38:02|0 |1 |1.0 |false |
ikXpPdr8nb5-jnprh5t5Og|Brant |3 |2017-10-01 16:30:30|0 |2 |0.9989294508082647|false |
vf1eAEnD39uPW4gF0m5hOg|Gloria |8 |2016-06-15 22:49:07|0 |1 |0.9958033616570202|false |
lzFlZcbx46sDLhY71byTSg|Hanna |1 |2013-07-02 22:19:12|0 |1 |0.9965851765618937|false |
8wHCGdXK4ImFJF9dfyn-zg|Oscar |1 |2017-10-28 19:44:28|0 |2 |0.9947491620742283|false |
jTt1Br3JKoA9oGG16cnlPg|Hanna |6 |2015-11-30 23:08:07|0 |1 |0.9965851765618937|false |
7KQZvs0HipXiUM_M1Y8kcw|Oscar |5 |2013-08-03 05:36:07|0 |2 |0.9947491620742283|false |
LkKg26Dp_QV0U4494vhTLw|Oscar |2 |2018-01-07 01:11:00|0 |2 |0.9947491620742283|false |
TtL9L8piP_l1WIKUTAn4iQ|Oscar |5 |2015-07-18 01:23:47|0 |2 |0.9947491620742283|false |
_uVmktQKPxEkkeKDHruTvw|Trina |2 |2017-10-15 22:24:02|0 |1 |0.9990773205388448|false |
SbsortZPBOIKnnJhK8wEVg|C. |1 |2015-02-02 15:31:06|0 |3 |1.0 |false |
6Bv6qg6F1zmYa17HxuJegg|Hope |2 |2017-12-11 03:00:35|0 |1 |0.994195395968719 |false |
cUAFFC7-asMIwipL8T68uw|Ruben |2 |2018-01-04 20:07:20|0 |2 |0.9942216419358822|false |
o9o7fnufIKa39KizUgXycQ|Monte |1 |2013-07-27 00:05:48|0 |2 |0.9648698804540158|false |
f_-0Hi_uFWhdih7oUVSNDQ|Oscar |2 |2018-02-25 21:42:05|0 |2 |0.9947491620742283|false |
g10uqGruj8OskJyG5IOOAQ|Malissa|2 |2017-10-19 23:29:13|0 |1 |1.0 |false |
+----------------------+-------+------------+-------------------+-----------+------+------------------+--------+
only showing top 20 rows

### Step 2a: How many users did we assign a gender to?

We could get a count of the number of users with each of the three possible values for `gender`, but we are going to start using some simple 
visualizations to do our profiling.

Run the following two cells which group and then `display` the results.  It may take a couple minutes to run.

In [0]:
df_gender_summary = df_user_gender.select("user_id", "gender").groupBy("gender").count()
df_gender_summary.show() 

+------+------+
gender| count|
+------+------+
 1|982924|
 3|205061|
 2|780718|
+------+------+

### Step 2b: Show a bar chart

In [0]:
display(df_gender_summary.orderBy("gender"))

gender count 1 982924 2 780718 3 205061

The above cell will generate a table, click on the `Plot Options...`` button to show the possible options; select the following:

* Key: gender
* Values: count
* Aggregation: SUM
* Display type: Bar chart
* Make sure the radio button for the type of bar chart is set to "grouped"

You may need to click on the bar chart option again to show the labels.

### Step 3: Fix the chart (iterate)
Did you chart come out in the order expected?  If not, go back and add a call to the `sort` method on the `df_gender_summary` DataFrame passed
to the `display` function (sort on the `gender` field).

### Step 4: Reducing the unknown users

We would want to get the count for the users with the `gender` value of 3 lower if we can.

If you go back and look at the data from the SSA, you will notice that the names always have the first letter capitalized.

What if we set both name values to be only lower case?

In the cell above where you define the `df_user_gender` DataFrame, modify the join condition, so that both `name` fields are compared based on their lower case 
values by ***enclosing them*** in a function named `LOWER()`.  Rerun that cell and the subsequent cells.  Did it reduce the number of users classified as 
3 for `gender`?  (the original value was 234,418).

### Step 5a: Do we know gender more often for elite users?
Since elite user are supposed to be tied to a real person (hopefully with a real, or at least realistic, name), 
possibly the percentage of elite users for whom we could not predict their gender would be lower.

Copy your code from step 2a above and include the `is_elite` field in your select clause. Since we would want to group by gender and whether the user was elite, 
also add the `is_elite` field to the `groupBy` method.

Earlier you added a call to the `sort` method, in this cell, add the call to the `sort` method at the end of your 
code for defining the new `df_elite_summary` DataFrame.

In [0]:
df_elite_summary = df_user_gender.select("user_id", "gender", "is_elite").\
groupBy("is_elite", "gender").count().orderBy("is_elite","gender")
df_elite_summary.show()

+--------+------+------+
is_elite|gender| count|
+--------+------+------+
 false| 1|938096|
 false| 2|754098|
 false| 3|200548|
 true| 1| 44828|
 true| 2| 26620|
 true| 3| 4513|
+--------+------+------+

### Step 5b: Display your results

Be sure to sort your result if you did not already do so when creating the DataFrame.

In [0]:
display(df_elite_summary)

is_elite gender count false 1 938096 false 2 754098 false 3 200548 true 1 44828 true 2 26620 true 3 4513

### Step 6a: Is the ratio of reviews similar to the ratio for users by gender?
We are going to use Spark SQL here, but below is the same query written in PySpark.

We have the review count for each user, so if we summed up that review count by gender, is the ratio of total reviews written (not necessarily in our dataset)
similar to the ratio we calculated above by gender?  Those users for which we could not predict a gender, do they write fewer reviews?  Is a lower percentage of the reviews written by those Yelpers we could not predict a gender for?

If we used used Spark SQL, we would need to use an aggregate function and GROUP BY clause in our query.  See the PySpark code below.

For PySpark, take a look at the <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy" target="_blank">groupBy</a> and 
the `agg` method used with `groupBy`.  You will also want to rename the calculated column using the <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.withColumnRenamed" target="_blank">withColumnRenamed</a> method.

In [0]:
#If you want to use Spark SQL
df_user_gender.createOrReplaceTempView("userGender")

df_review_summary = spark.sql("""
SELECT gender, SUM(review_count) AS total_reviews
FROM userGender
GROUP BY gender
ORDER BY gender
""")
df_review_summary.show()

+------+-------------+
gender|total_reviews|
+------+-------------+
 1| 22211609|
 2| 16881150|
 3| 4552006|
+------+-------------+

In [0]:
# If you want to use PySpark:
df_review_summary = df_user_gender.select("gender","review_count").\
groupBy("gender").agg({"review_count":"sum"}).\
withColumnRenamed("sum(review_count)","total_reviews").sort("gender")
df_review_summary.show()

+------+-------------+
gender|total_reviews|
+------+-------------+
 1| 22211609|
 2| 16881150|
 3| 4552006|
+------+-------------+

### Step 6b: Display your results

The following cell displays your results.

In [0]:
display(df_review_summary)

gender total_reviews 1 22211609 2 16881150 3 4552006

# Part 3: Trends and Distributions ...

In this part of the notebook, you will be exploring The gender breakdown of users joining Yelp.  To do that, we included the `yelping_since` field 
in the `df_users` DataFrame earlier in this exercise.


### Step 1a: Area Chart
A question we might have about our users is how many women, men, and unknowns (people who's gender we could not predict) joined Yelp each year.  Good 
options for this might be the area plot or the line graph. Here we will get the the year from the `yelping_since` field you just added (there is a `YEAR()` function
in SQL), and count the number of users for each year and gender.  Here the results are sorted by gender and year.

Your result will have three columns: gender, yelp_year, and user_count 

### Step 1b: Display your area chart
Select the following options (you may have given your fields different aliases):

* Chart type: Area
* Keys: yelp_year
* Series groupings: gender
* Values: user_count
* Aggregation: SUM
* radio buttons: Stacked

In [0]:
df_user_gender.createOrReplaceTempView("genderSummary")

df_genderByYear = spark.sql("""
SELECT gender, YEAR(yelping_since) AS yelp_year, COUNT(user_id) AS user_count
FROM genderSummary
GROUP BY gender, YEAR(yelping_since)
ORDER BY gender, YEAR(yelping_since)
""").cache()
df_genderByYear.show(50)

+------+---------+----------+
gender|yelp_year|user_count|
+------+---------+----------+
 1| 2004| 21|
 1| 2005| 496|
 1| 2006| 2506|
 1| 2007| 7525|
 1| 2008| 15123|
 1| 2009| 27858|
 1| 2010| 50469|
 1| 2011| 89105|
 1| 2012| 97934|
 1| 2013| 108860|
 1| 2014| 128302|
 1| 2015| 138184|
 1| 2016| 128692|
 1| 2017| 84683|
 1| 2018| 64390|
 1| 2019| 38776|
 2| 2004| 57|
 2| 2005| 395|
 2| 2006| 2723|
 2| 2007| 7326|
 2| 2008| 14781|
 2| 2009| 32003|
 2| 2010| 52335|
 2| 2011| 76743|
 2| 2012| 84607|
 2| 2013| 89680|
 2| 2014| 97960|
 2| 2015| 102470|
 2| 2016| 88826|
 2| 2017| 57802|
 2| 2018| 44918|
 2| 2019| 28092|
 3| 2004| 4|
 3| 2005| 131|
 3| 2006| 823|
 3| 2007| 2304|
 3| 2008| 4423|
 3| 2009| 8453|
 3| 2010| 12302|
 3| 2011| 19228|
 3| 2012| 20639|
 3| 2013| 22840|
 3| 2014| 24565|
 3| 2015| 26613|
 3| 2016| 23896|
 3| 2017| 16396|
 3| 2018| 13584|
 3| 2019| 8860|
+------+---------+----------+

### Step 1b: Display your area chart
If we want to see the composition of the total number of users joining each year, a stacked area chart would be a good option.  Select the chart options to do the following:

* Chart type: Area (stacked)
* X-Axis: these should be the 4-digit year
* There should be a stacked group for each gender

In [0]:
display(df_genderByYear)

gender yelp_year user_count 1 2004 21 1 2005 496 1 2006 2506 1 2007 7525 1 2008 15123 1 2009 27858 1 2010 50469 1 2011 89105 1 2012 97934 1 2013 108860 1 2014 128302 1 2015 138184 1 2016 128692 1 2017 84683 1 2018 64390 1 2019 38776 2 2004 57 2 2005 395 2 2006 2723 2 2007 7326 2 2008 14781 2 2009 32003 2 2010 52335 2 2011 76743 2 2012 84607 2 2013 89680 2 2014 97960 2 2015 102470 2 2016 88826 2 2017 57802 2 2018 44918 2 2019 28092 3 2004 4 3 2005 131 3 2006 823 3 2007 2304 3 2008 4423 3 2009 8453 3 2010 12302 3 2011 19228 3 2012 20639 3 2013 22840 3 2014 24565 3 2015 26613 3 2016 23896 3 2017 16396 3 2018 13584 3 2019 8860

### Step 2: Line Chart
If we wanted to compare the totals by gender for each year, the line chart would be a better alternative. In the next cell, use the `display()` function again, but generate a line chart using the same `df_genderByYear` DataFrame.

In [0]:
display(df_genderByYear)

gender yelp_year user_count 1 2004 21 1 2005 496 1 2006 2506 1 2007 7525 1 2008 15123 1 2009 27858 1 2010 50469 1 2011 89105 1 2012 97934 1 2013 108860 1 2014 128302 1 2015 138184 1 2016 128692 1 2017 84683 1 2018 64390 1 2019 38776 2 2004 57 2 2005 395 2 2006 2723 2 2007 7326 2 2008 14781 2 2009 32003 2 2010 52335 2 2011 76743 2 2012 84607 2 2013 89680 2 2014 97960 2 2015 102470 2 2016 88826 2 2017 57802 2 2018 44918 2 2019 28092 3 2004 4 3 2005 131 3 2006 823 3 2007 2304 3 2008 4423 3 2009 8453 3 2010 12302 3 2011 19228 3 2012 20639 3 2013 22840 3 2014 24565 3 2015 26613 3 2016 23896 3 2017 16396 3 2018 13584 3 2019 8860

### Step 3: Grouped Bar Chart
In the following cell, display the same `df_genderByYear` DataFrame, but create a grouped bar chart where the year is on the x-axis, the user_count is on the y-axis, 
and for each year there are 3 bars - one for each of the 3 possible gender values.

In [0]:
display(df_genderByYear)

gender yelp_year user_count 1 2004 21 1 2005 496 1 2006 2506 1 2007 7525 1 2008 15123 1 2009 27858 1 2010 50469 1 2011 89105 1 2012 97934 1 2013 108860 1 2014 128302 1 2015 138184 1 2016 128692 1 2017 84683 1 2018 64390 1 2019 38776 2 2004 57 2 2005 395 2 2006 2723 2 2007 7326 2 2008 14781 2 2009 32003 2 2010 52335 2 2011 76743 2 2012 84607 2 2013 89680 2 2014 97960 2 2015 102470 2 2016 88826 2 2017 57802 2 2018 44918 2 2019 28092 3 2004 4 3 2005 131 3 2006 823 3 2007 2304 3 2008 4423 3 2009 8453 3 2010 12302 3 2011 19228 3 2012 20639 3 2013 22840 3 2014 24565 3 2015 26613 3 2016 23896 3 2017 16396 3 2018 13584 3 2019 8860

### Step 4: Histogram
What if we wanted to know the distribution as to the number of reviews users wrote (not necesarily that are in our dataset).
One approach is the histogram, but keep in mind it will only look at the first 1,000 records, so we will need to narrow our range since we know we have a user with over 14,000 records.
In the following query we get just the `review_count` field, but we cap the count at 100, so our user with over 14,000 reviews will instead have a review count (for this query), of 100.

In the current version of Databricks there are some issues with generating the histogram.  Select the chart type from the chart drop-down list, not from within the plot options, and you will be stuck with the default of 20 bins.  Since
in this case we capped the review count at 100, each bin will be a range of 5 (since we have a range of 100 and we divide it evenly into 20 bins).

In [0]:
df_cappedReviewsCount = spark.sql("""
SELECT IF(review_count > 100, 100, review_count) AS capped_count
FROM userGender
WHERE review_count > 0
ORDER BY capped_count
""").cache()
df_cappedReviewsCount.show()

+------------+
capped_count|
+------------+
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
+------------+
only showing top 20 rows

#### Step 4b: Displaying a Histogram

In the following cell display the histogram.  Due to limitations in the current version of the software, you need to select the histogram from the drop-down list (don't start with a bar chart).  Does the cap you placed in step 4 seem reasonable?

In [0]:
display(df_cappedReviewsCount)

capped_count 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 Showing the first 1000 rows.